In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os

# DataSet

In [3]:
import random
import gzip, struct

class DataSet:
    batch_index = 0
    
    def __init__(self, data_dir, batch_size = None, one_hot = False, seed = 0):
        self.data_dir = data_dir
        X, Y = self.read()
        shape = X.shape
        X = X.reshape([shape[0], shape[1] * shape[2]])
        self.X = X.astype(np.float)/255
        self.size = self.X.shape[0]
        if batch_size == None:
            self.batch_size = self.size
        else:
            self.batch_size = batch_size
        # abandom last few samples
        self.batch_num = int(self.size / self.batch_size)
        # shuffle samples
        np.random.seed(seed)
        np.random.shuffle(self.X)
        np.random.seed(seed)
        np.random.shuffle(Y)
        self.one_hot = one_hot
        if one_hot:
            y_vec = np.zeros((len(Y), 10), dtype=np.float)
            for i, label in enumerate(Y):
                y_vec[i, Y[i]] = 1.0
            self.Y = y_vec
        else:
            self.Y = Y
    
    def read(self):
        with gzip.open(self.data_dir['Y']) as flbl:
            magic, num = struct.unpack(">II", flbl.read(8))
            label = np.fromstring(flbl.read(), dtype=np.int8)
        with gzip.open(self.data_dir['X'], 'rb') as fimg:
            magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
            image = np.fromstring(fimg.read(), dtype=np.uint8).reshape(len(label), rows, cols)
        return image,label
    
    def next_batch(self):
        start = self.batch_index * self.batch_size
        end = (self.batch_index + 1) * self.batch_size
        self.batch_index = (self.batch_index + 1) % self.batch_num
        if self.one_hot:
            return self.X[start:end, :], self.Y[start:end, :]
        else:
            return self.X[start:end, :], self.Y[start:end]
        
    def sample_batch(self):
        index = random.randrange(self.batch_num)
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        if self.one_hot:
            return self.X[start:end, :], self.Y[start:end, :]
        else:
            return self.X[start:end, :], self.Y[start:end]

In [4]:
def test_DataSet():
    train_dir = {
        'X': './mnist/train-images-idx3-ubyte.gz', 
        'Y': './mnist/train-labels-idx1-ubyte.gz'
    }
    test_dir = {
        'X': './mnist/t10k-images-idx3-ubyte.gz', 
        'Y': './mnist/t10k-labels-idx1-ubyte.gz'
    }
    mnist = DataSet(test_dir, 2)
    print('batch index: %d' % mnist.batch_index)
    X, Y = mnist.next_batch()
    print('batch index: %d' % mnist.batch_index)
    print('X:')
    print(X)
    print('Y:')
    print(Y)

In [5]:
test_DataSet()

batch index: 0
batch index: 1
X:
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Y:
[8 7]


In [6]:
train_dir = {
    'X': './mnist/train-images-idx3-ubyte.gz', 
    'Y': './mnist/train-labels-idx1-ubyte.gz'
}

In [7]:
test_dir = {
    'X': './mnist/t10k-images-idx3-ubyte.gz', 
    'Y': './mnist/t10k-labels-idx1-ubyte.gz'
}

# Operation

In [8]:
def weight(shape, name='weights'):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name=name)

In [9]:
def bias(shape, name='biases'):
    return tf.Variable(tf.constant(0.1, shape=shape), name=name)

# Model

In [10]:
class RBM:
    i = 0 # fliping index for computing pseudo likelihood
    
    def __init__(self, n_visible=784, n_hidden=500, k=30, momentum=False):
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.k = k
        
        self.lr = tf.placeholder(tf.float32)
        if momentum:
            self.momentum = tf.placeholder(tf.float32)
        else:
            self.momentum = 0.0
        self.w = weight([n_visible, n_hidden], 'w')
        self.hb = bias([n_hidden], 'hb')
        self.vb = bias([n_visible], 'vb')
        
        self.w_v = tf.Variable(tf.zeros([n_visible, n_hidden]), dtype=tf.float32)
        self.hb_v = tf.Variable(tf.zeros([n_hidden]), dtype=tf.float32)
        self.vb_v = tf.Variable(tf.zeros([n_visible]), dtype=tf.float32)
        
    def propup(self, visible):
        pre_sigmoid_activation = tf.matmul(visible, self.w) + self.hb
        return tf.nn.sigmoid(pre_sigmoid_activation)
    
    def propdown(self, hidden):
        pre_sigmoid_activation = tf.matmul(hidden, tf.transpose(self.w)) + self.vb
        return tf.nn.sigmoid(pre_sigmoid_activation)
    
    def sample_h_given_v(self, v_sample):
        h_props = self.propup(v_sample)
        h_sample = tf.nn.relu(tf.sign(h_props - tf.random_uniform(tf.shape(h_props))))
        return h_sample
    
    def sample_v_given_h(self, h_sample):
        v_props = self.propdown(h_sample)
        v_sample = tf.nn.relu(tf.sign(v_props - tf.random_uniform(tf.shape(v_props))))
        return v_sample
    
    def CD_k(self, visibles):       
        # k steps gibbs sampling
        v_samples = visibles
        h_samples = self.sample_h_given_v(v_samples)
        for i in range(self.k):
            v_samples = self.sample_v_given_h(h_samples)
            h_samples = self.sample_h_given_v(v_samples)
        
        h0_props = self.propup(visibles)
        w_positive_grad = tf.matmul(tf.transpose(visibles), h0_props)
        w_negative_grad = tf.matmul(tf.transpose(v_samples), h_samples)
        w_grad = (w_positive_grad - w_negative_grad) / tf.to_float(tf.shape(visibles)[0])
        hb_grad = tf.reduce_mean(h0_props - h_samples, 0)
        vb_grad = tf.reduce_mean(visibles - v_samples, 0)
        return w_grad, hb_grad, vb_grad
    
    def learn(self, visibles):
        w_grad, hb_grad, vb_grad = self.CD_k(visibles)
        # compute new velocities
        new_w_v = self.momentum * self.w_v + self.lr * w_grad
        new_hb_v = self.momentum * self.hb_v + self.lr * hb_grad
        new_vb_v = self.momentum * self.vb_v + self.lr * vb_grad
        # update parameters
        update_w = tf.assign(self.w, self.w + new_w_v)
        update_hb = tf.assign(self.hb, self.hb + new_hb_v)
        update_vb = tf.assign(self.vb, self.vb + new_vb_v)
        # update velocities
        update_w_v = tf.assign(self.w_v, new_w_v)
        update_hb_v = tf.assign(self.hb_v, new_hb_v)
        update_vb_v = tf.assign(self.vb_v, new_vb_v)
        
        return [update_w, update_hb, update_vb, update_w_v, update_hb_v, update_vb_v]
        
    def sampler(self, visibles, steps=5000):
        v_samples = visibles
        for step in range(steps):
            v_samples = self.sample_v_given_h(self.sample_h_given_v(v_samples))
        return v_samples
    
    def free_energy(self, visibles):
        first_term = tf.matmul(visibles, tf.reshape(self.vb, [tf.shape(self.vb)[0], 1]))
        second_term = tf.reduce_sum(tf.log(1 + tf.exp(self.hb + tf.matmul(visibles, self.w))), axis=1)
        return - first_term - second_term
    
    def pseudo_likelihood(self, visibles):
        x = tf.round(visibles)
        x_fe = self.free_energy(x)
        split0, split1, split2 = tf.split(x, [self.i, 1, tf.shape(x)[1] - self.i - 1], 1)
        xi = tf.concat([split0, 1 - split1, split2], 1)
        self.i = (self.i + 1) % self.n_visible
        xi_fe = self.free_energy(xi)
        return tf.reduce_mean(self.n_visible * tf.log(tf.nn.sigmoid(xi_fe - x_fe)), axis=0)

![](markov_chain.png)

# Train

In [11]:
import scipy.misc

# 保存图片
def save_images(images, size, path):
    
    """
    Save the samples images
    The best size number is
            int(max(sqrt(image.shape[0]),sqrt(image.shape[1]))) + 1
    example:
        The batch_size is 64, then the size is recommended [8, 8]
        The batch_size is 32, then the size is recommended [6, 6]
    """
    
    # 图片归一化，主要用于生成器输出是tanh形式的归一化
    img = (images + 1.0) / 2.0
    h, w = img.shape[1], img.shape[2]
    
    # 生成一个大画布，用来保存生成的batch_size个图像
    merge_img = np.zeros((h * size[0], w * size[1]))
    
    # 循环把画布各个位置的值赋为batch里各幅图像的值
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        merge_img[j*h:j*h+h, i*w:i*w+w] = image
        
    # 保存画布
    return scipy.misc.imsave(path, merge_img)

In [12]:
def train(train_data, epoches):
    logs_dir = './logs'
    samples_dir = './samples'
    
    x = tf.placeholder(tf.float32, shape=[None, 784])
    noise_x, _ = train_data.sample_batch()
    # noise_x = tf.random_normal([train_data.batch_size, 784])
    rbm = RBM()
    step = rbm.learn(x)
    sampler = rbm.sampler(x)
    pl = rbm.pseudo_likelihood(x)
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        mean_cost = []
        epoch = 1
        for i in range(epoches * train_data.batch_num):
            # draw samples
            if i % 500 == 0:
                samples = sess.run(sampler, feed_dict = {x: noise_x})
                samples = samples.reshape([train_data.batch_size, 28, 28])
                save_images(samples, [8, 8], os.path.join(samples_dir, 'iteration_%d.png' % i))
                print('Saved samples.')
            batch_x, _ = train_data.next_batch()
            sess.run(step, feed_dict = {x: batch_x, rbm.lr: 0.1})
            cost = sess.run(pl, feed_dict = {x: batch_x})
            mean_cost.append(cost)
            # save model
            if i is not 0 and train_data.batch_index is 0:
                checkpoint_path = os.path.join(logs_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step = epoch + 1)
                print('Saved Model.')
            # print pseudo likelihood
            if i is not 0 and train_data.batch_index is 0:
                print('Epoch %d Cost %g' % (epoch, np.mean(mean_cost)))
                mean_cost = []
                epoch += 1
        print('Test')
        samples = sess.run(sampler, feed_dict = {x: noise_x})
        samples = samples.reshape([train_data.batch_size, 28, 28])
        save_images(samples, [8, 8], os.path.join(samples_dir, 'test.png'))
        print('Saved samples.')

In [13]:
train_data = DataSet(data_dir=train_dir, batch_size=64, one_hot=True)

In [14]:
train(train_data, 50)

Saved samples.
Saved samples.
Saved Model.
Epoch 1 Cost -1.92544
Saved samples.
Saved samples.
Saved Model.
Epoch 2 Cost -0.341011
Saved samples.
Saved samples.
Saved Model.
Epoch 3 Cost -0.216929
Saved samples.
Saved samples.
Saved Model.
Epoch 4 Cost -0.148479
Saved samples.
Saved samples.
Saved Model.
Epoch 5 Cost -0.121412
Saved samples.
Saved samples.
Saved Model.
Epoch 6 Cost -0.106842
Saved samples.
Saved samples.
Saved Model.
Epoch 7 Cost -0.0951367
Saved samples.
Saved Model.
Epoch 8 Cost -0.0763526
Saved samples.
Saved samples.
Saved Model.
Epoch 9 Cost -0.0658732
Saved samples.
Saved samples.
Saved Model.
Epoch 10 Cost -0.0554306
Saved samples.
Saved samples.
Saved Model.
Epoch 11 Cost -0.0514627
Saved samples.
Saved samples.
Saved Model.
Epoch 12 Cost -0.0426717
Saved samples.
Saved samples.
Saved Model.
Epoch 13 Cost -0.0426232
Saved samples.
Saved samples.
Saved Model.
Epoch 14 Cost -0.0357415
Saved samples.
Saved samples.
Saved Model.
Epoch 15 Cost -0.0306901
Saved sampl

# Reference

1. [GitHub: tensorflow-rbm](https://github.com/meownoid/tensorfow-rbm)
2. [Theano: RBM](http://deeplearning.net/tutorial/rbm.html#rbm)
3. [Stackoverflow: RBM implementation](http://stackoverflow.com/questions/34760981/rbm-implementation-with-tensorflow/35446666#35446666)